<a href="https://colab.research.google.com/github/kmkawa/podcast-book-recommender/blob/main/scrape_spotify_podcast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Getting Podcast Data using Spotify API



In [ ]:
pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.9/339.9 kB 9.4 MB/s eta 0:00:00


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#ADD PERSONAL CLIENT ID AND SECRET

In [ ]:
#This cell gets show titles by searching for things and then just taking them.

#when I use this one, I get ~500 shows
def get_shows_from_query(query, limit=50):
    results = sp.search(q=query, type="show", limit=limit)
    shows = results["shows"]["items"]
    return [show["name"] for show in shows]

#this one can return ~4500 shows
def search_shows(query, pages=10):
    titles = []
    for i in range(pages):
        results = sp.search(
            q=query,
            type="show",
            limit=50,
            offset=i * 50
        )
        for show in results["shows"]["items"]:
            titles.append(show["name"])
    return titles

#just using random key works to try to get diverse range of podcast hits
queries = [
    "a", "the", "podcast", "show",
    "news", "crime", "history", "science",
    "life", "business", "sports", "tech"
]

titles = set()

for q in queries:
    for title in search_shows(q):
        titles.add(title)

print(f"Found {len(titles)} unique podcast titles.")
print(list(titles)[:20])

top_podcasts = titles


Found 4578 unique podcast titles.
['Mind of a Monster: The Killer Nurse', 'Truth For Life Daily Devotions', 'Entiende Tu Mente', 'Frugal Friends Podcast', 'The Jeremiah Show: Pop Culture, Music Icons Food Gods', 'The Sports Med Bros Podcast', 'Historically Badass Broads', 'Francis Foster Sorts Your Life Out', 'The Stories of Mahabharata', 'Business With The Boyz', '出塊True Crime', '安住紳一郎の日曜天国', '菇武門Podcast——何韻詩、黃詠詩', 'LIVES', 'Crime World Presents: Groomed', 'Felon True Crime', 'The Big Honker Podcast', 'The Heffernan Show (King Of Queens Podcast)', 'Consider This from NPR', 'True Crime en Español, Historias de Crimen, Historias Reales, True Crime Español']


In [ ]:
from google.colab import drive
import os

# Mount Drive
drive.mount('/content/drive')

# Create folder if it doesn't exist
SAVE_FOLDER = "/content/drive/My Drive"
os.makedirs(SAVE_FOLDER, exist_ok=True)

# Save path
SAVE_FILE = os.path.join(SAVE_FOLDER, "spotify_podcast_data.json")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Here we take the titles (that were gotten from SPotify, therefore should all be there)
# And use the API to grab the description and the number of episodes (I tried to get the ave length of the shows, but that took a lot of time)

import base64
import requests
import json
import time
import os
from tqdm import tqdm

# ========== CONFIGURATION ==========
CLIENT_ID = "your_client_id_here"
CLIENT_SECRET = "your_client_secret_here"

SAVE_FILE = "/content/drive/MyDrive/spotify_podcast_data.json"
BATCH_SIZE = 100
RETRY_LIMIT = 5
RETRY_DELAY = 2
THROTTLE_DELAY = 0.2

BASE_URL = "https://api.spotify.com/v1"

# ========== AUTHENTICATION (CLIENT CREDENTIALS FLOW) ==========
def get_spotify_token(client_id, client_secret):
    """Retrieve an access token using client credentials."""
    auth_str = f"{client_id}:{client_secret}"
    auth_bytes = base64.b64encode(auth_str.encode())
    headers = {"Authorization": f"Basic {auth_bytes.decode()}"}
    data = {"grant_type": "client_credentials"}

    res = requests.post("https://accounts.spotify.com/api/token", headers=headers, data=data)
    if res.status_code == 200:
        return res.json()["access_token"]
    else:
        raise Exception(f"Failed to authenticate: {res.text}")

# Get the token
token = get_spotify_token(client_id, client_secret)
headers = {"Authorization": f"Bearer {token}"}

# ========== SPOTIFY API HELPERS ==========
def safe_request(url, params=None, headers=headers):
    """Requests with retry logic"""
    for attempt in range(RETRY_LIMIT):
        try:
            r = requests.get(url, headers=headers, params=params)
            if r.status_code == 200:
                return r.json()
            elif r.status_code == 429:
                delay = int(r.headers.get("Retry-After", 10))
                print(f"Rate limited. Sleeping for {delay}s...")
                time.sleep(delay)
            else:
                print(f"Unexpected response {r.status_code}: {r.text}")
        except Exception as e:
            print(f"Error: {e}")

        time.sleep(RETRY_DELAY)
    return None

def search_podcast_by_title(title):
    url = f"{BASE_URL}/search"
    params = {"q": title, "type": "show", "limit": 1}
    return safe_request(url, params)

def get_show_details(show_id):
    url = f"{BASE_URL}/shows/{show_id}"
    return safe_request(url)

# ========== MAIN SCRAPER FUNCTION ==========
def fetch_podcast_metadata(titles):
    # Load or initialize data store
    if os.path.exists(SAVE_FILE):
        with open(SAVE_FILE, "r") as f:
            podcast_data = json.load(f)
        print(f"Loaded existing data: {len(podcast_data)} entries.")
    else:
        podcast_data = {}

    # Iterate over titles
    for i, title in enumerate(tqdm(titles)):
        if title in podcast_data:
            continue

        # Fetch podcast details
        search_res = search_podcast_by_title(title)
        if search_res and search_res["shows"]["items"]:
            show = search_res["shows"]["items"][0]
            show_id = show["id"]
            time.sleep(THROTTLE_DELAY)

            details = get_show_details(show_id)
            if details:
                podcast_data[title] = {
                    "description": details.get("description", "").strip(),
                    "total_episodes": details.get("total_episodes", 0),
                }
        else:
            podcast_data[title] = {"description": None, "total_episodes": None}

        # Save periodically
        if (i + 1) % BATCH_SIZE == 0:
            with open(SAVE_FILE, "w") as f:
                json.dump(podcast_data, f, indent=2)
            print(f"Saved progress: {i + 1} titles.")

    # Final save
    with open(SAVE_FILE, "w") as f:
        json.dump(podcast_data, f, indent=2)
    print("Scraping complete!")

# ========== RUN THE SCRAPER ==========
fetch_podcast_metadata(titles)


  2%|▏         | 100/4578 [01:27<58:15,  1.28it/s] 

Saved progress: 100 titles.


  4%|▍         | 200/4578 [02:59<1:02:58,  1.16it/s]

Saved progress: 200 titles.


  7%|▋         | 300/4578 [04:39<1:17:47,  1.09s/it]

...

Unexpected response 401: {
  "error": {
    "status": 401,
    "message": "The access token expired"
  }
}
Unexpected response 401: {
  "error": {
    "status": 401,
    "message": "The access token expired"
  }
}
Unexpected response 401: {
  "error": {
    "status": 401,
    "message": "The access token expired"
  }
}
Unexpected response 401: {
  "error": {
    "status": 401,
    "message": "The access token expired"
  }
}
Unexpected response 401: {
  "error": {
    "status": 401,
    "message": "The access token expired"
  }
}


 91%|█████████ | 4164/4578 [1:20:23<1:10:00, 10.15s/it]

Unexpected response 401: {
  "error": {
    "status": 401,
    "message": "The access token expired"
  }
}
Unexpected response 401: {
  "error": {
    "status": 401,
    "message": "The access token expired"
  }
}
Unexpected response 401: {
  "error": {
    "status": 401,
    "message": "The access token expired"
  }
}
Unexpected response 401: {
  "error": {
    "status": 401,
    "message": "The access token expired"
  }
}
Unexpected response 401: {
  "error": {
    "status": 401,
    "message": "The access token expired"
  }
}


 91%|█████████ | 4164/4578 [1:20:32<08:00,  1.16s/it]  


KeyboardInterrupt: 

In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=f1e77b2ab6cdf9cf885c582d9afedf66c4cb549263210242201acbef6c54a319
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [ ]:
import json
from langdetect import detect, LangDetectException

# Load the raw data
input_file = "/content/drive/My Drive/spotify_podcast_data.json"
output_file = "/content/drive/My Drive/spotify_podcast_data_cleaned.json"

# Load JSON
with open(input_file, "r") as f:
    data = json.load(f)

total_count = len(data)
cleaned_data = {}
removed_count = 0
error_count = 0
none_desc_count = 0

for title, metadata in data.items():
    try:
        title_lang = detect(title)

        description = metadata.get("description", "")
        if not isinstance(description, str):
            none_desc_count += 1
            description = ""   # Default to empty string if description is None

        desc_lang = detect(description) if description.strip() else ""  # Skip detection on empty

        # Keep entries where title and description (if present) are English
        if title_lang == "en" and (desc_lang in ["en", ""]):
            cleaned_data[title] = metadata
        else:
            removed_count += 1

    except LangDetectException:
        error_count += 1

# Save cleaned file
with open(output_file, "w") as f:
    json.dump(cleaned_data, f, indent=2)

kept_count = len(cleaned_data)

# Print summary
print(f"Total entries processed: {total_count}")
print(f"Removed (non-English or skipped): {removed_count}")
print(f"Skipped due to detection errors: {error_count}")
print(f"Entries with no description: {none_desc_count}")
print(f"Remaining (English only): {kept_count}")
print(f"✔️ Cleaned dataset saved to: {output_file}")


🎯 Cleanup Summary
------------------
Total entries processed: 4099
Removed (non-English or skipped): 1164
Skipped due to detection errors: 45
Entries with no description: 57
Remaining (English only): 2890
✔️ Cleaned dataset saved to: /content/drive/My Drive/spotify_podcast_data_cleaned.json


In [ ]:
import json
from langdetect import detect, LangDetectException

# Load your raw data
input_file = "/content/drive/MyDrive/DIS Work/Core Course Work/podcast_data_small.json"  # <-- Update path if needed
output_file = "/content/drive/MyDrive/DIS Work/Core Course Work/podcast_data_small_cleaned.json"

# Load JSON
with open(input_file, "r") as f:
    data = json.load(f)

total_count = len(data)
cleaned_data = {}
removed_count = 0
error_count = 0
none_desc_count = 0

for title, metadata in data.items():
    try:
        title_lang = detect(title)

        description = metadata.get("description", "")
        if not isinstance(description, str):
            none_desc_count += 1
            description = ""   # Default to empty string if description is None

        desc_lang = detect(description) if description.strip() else ""  # Skip detection on empty

        # Keep entries where title and description (if present) are English
        if title_lang == "en" and (desc_lang in ["en", ""]):
            cleaned_data[title] = metadata
        else:
            removed_count += 1

    except LangDetectException:
        error_count += 1

# Save cleaned file
with open(output_file, "w") as f:
    json.dump(cleaned_data, f, indent=2)

kept_count = len(cleaned_data)

# Print summary
print(f"Total entries processed: {total_count}")
print(f"Removed (non-English or skipped): {removed_count}")
print(f"Skipped due to detection errors: {error_count}")
print(f"Entries with no description: {none_desc_count}")
print(f"Remaining (English only): {kept_count}")
print(f"✔️ Cleaned dataset saved to: {output_file}")


🎯 Cleanup Summary
------------------
Total entries processed: 503
Removed (non-English or skipped): 138
Skipped due to detection errors: 8
Entries with no description: 0
Remaining (English only): 357
✔️ Cleaned dataset saved to: /content/drive/MyDrive/DIS Work/Core Course Work/podcast_data_small_cleaned.json
